# Hard Questions



1. https://datalemur.com/questions/user-retention

Assume you're given a table containing information on Facebook user actions. Write a query to obtain number of monthly active users (MAUs) in July 2022, including the month in numerical format "1, 2, 3".

Hint:
- An active user is defined as a user who has performed actions such as 'sign-in', 'like', or 'comment' in both the current month and the previous month.

`user_actions` Table:
| Column Name | Type |
| --- | --- |
| user_id | integer |
| event_id | integer |
| event_type | string ("sign-in, "like", "comment") |
| event_date | datetime |

```sql
WITH cte as (SELECT user_id, month, 
    row_number() OVER (
    PARTITION BY user_id
    ORDER BY month
    )
FROM
    (SELECT DISTINCT 
    user_id, 
    EXTRACT(MONTH FROM event_date) as month
    FROM user_actions
    WHERE event_date BETWEEN '2022-06-01' AND '2022-07-31'
    ORDER BY user_id) as subquery)

SELECT month, COUNT(max) as monthly_active_users
FROM
(SELECT month, user_id, MAX(row_number)
FROM cte
GROUP BY month, user_id
HAVING MAX(row_number) = 2) AS SUB2
GROUP BY month
```


2. https://datalemur.com/questions/yoy-growth-rate

Assume you're given a table containing information about Wayfair user transactions for different products. Write a query to calculate the year-on-year growth rate for the total spend of each product, grouping the results by product ID.

The output should include the year in ascending order, product ID, current year's spend, previous year's spend and year-on-year growth percentage, rounded to 2 decimal places.

`user_transactions` Table:
| Column Name | Type |
| --- | --- |
| transaction_id | integer |
| product_id | integer |
| spend | decimal |
| transaction_date | datetime |

```sql
SELECT *,
  (CASE WHEN prev_year_spend IS NOT NULL 
  THEN ROUND(100.0*(curr_year_spend - prev_year_spend)/prev_year_spend,2)
  ELSE NULL
  END) AS yoy_rate
FROM
(SELECT 
  EXTRACT(YEAR FROM transaction_date) as year,
  product_id,
  spend as curr_year_spend,
  LAG(spend) OVER (
    PARTITION BY product_id
    ORDER BY EXTRACT(YEAR FROM transaction_date)
  ) as prev_year_spend
FROM user_transactions) AS subquery
```


3. https://datalemur.com/questions/prime-warehouse-storage

Amazon wants to maximize the storage capacity of its 500,000 square-foot warehouse by prioritizing a specific batch of prime items. The specific prime product batch detailed in the inventory table must be maintained.

So, if the prime product batch specified in the item_category column included 1 laptop and 1 side table, that would be the base batch. We could not add another laptop without also adding a side table; they come all together as a batch set.

After prioritizing the maximum number of prime batches, any remaining square footage will be utilized to stock non-prime batches, which also come in batch sets and cannot be separated into individual items.

Write a query to find the maximum number of prime and non-prime batches that can be stored in the 500,000 square feet warehouse based on the following criteria:
- Prioritize stocking prime batches
- After accommodating prime items, allocate any remaining space to non-prime batches

Output the item_type with prime_eligible first followed by not_prime, along with the maximum number of batches that can be stocked.

Assumptions:
- Again, products must be stocked in batches, so we want to find the largest available quantity of prime batches, and then the largest available quantity of non-prime batches
- Non-prime items must always be available in stock to meet customer demand, so the non-prime item count should never be zero.
- Item count should be whole numbers (integers).

`inventory` table:
| Column Name | Type |
| --- | --- |
| item_id | integer |
| item_type | string |
| item_category | string |
| square_footage | decimal |

```sql
with cte AS 
    (SELECT item_type, 
    SUM(square_footage), COUNT(item_id) AS batch_item_count,
    LAG(sum(square_footage)) OVER(
    PARTITION BY 1
    ORDER BY  (CASE WHEN item_type = 'prime_eligible' THEN 1 ELSE 2 END)
    )
    FROM inventory
    GROUP BY item_type)

SELECT * 
FROM 
    (SELECT item_type, FLOOR(500000/sum) * batch_item_count  AS item_count
    FROM cte
    WHERE item_type = 'prime_eligible'
    UNION
    SELECT item_type, FLOOR((500000%lag)/sum) * batch_item_count AS item_count
    FROM cte
    WHERE item_type != 'prime_eligible') as sq
ORDER BY CASE WHEN item_type = 'prime_eligible' THEN 1 ELSE 2 END
```


# Medium Questions

1. https://datalemur.com/questions/sql-third-transaction

Assume you are given the table below on Uber transactions made by users. Write a query to obtain the third transaction of every user. Output the user id, spend and transaction date.

`transactions` Table:
 | Column Name | Type |
 | --- | --- |
 | user_id | integer |
 | spend | decimal |
 | transaction_date | timestamp |

```sql
SELECT user_id, spend, transaction_date
FROM
(SELECT *,
  RANK() OVER(
    PARTITION BY user_id
    ORDER BY transaction_date
  ) AS trans_order
FROM transactions) as subquery
WHERE trans_order = 3
```

2. https://datalemur.com/questions/time-spent-snaps

Assume you're given tables with information on Snapchat users, including their ages and time spent sending and opening snaps.

Write a query to obtain a breakdown of the time spent sending vs. opening snaps as a percentage of total time spent on these activities grouped by age group. Round the percentage to 2 decimal places in the output.

Notes:

- Calculate the following percentages:
    - time spent sending / (Time spent sending + Time spent opening)
    - Time spent opening / (Time spent sending + Time spent opening)
- To avoid integer division in percentages, multiply by 100.0 and not 100.

`activities` Table
| Column Name | Type |
| --- | --- |
| activity_id | integer |
| user_id | integer |
| activity_type | string ('send', 'open', 'chat') |
| time_spent | float |
| activity_date | datetime |

`age_breakdown` Table
| Column Name | Type |
| --- | --- |
| user_id | integer |
| age_bucket | string ('21-25', '26-30', '31-25') |

```sql
WITH cte as 
    (SELECT act.user_id, act.activity_type , act.time_spent, ab.age_bucket
     FROM activities as act
     JOIN age_breakdown as ab
     ON act.user_id = ab.user_id
     WHERE act.activity_type in ('send','open'))

SELECT 
    age_bucket,
    ROUND(SUM(CASE WHEN activity_type = 'send' THEN time_spent ELSE 0 END) /
    SUM(CASE WHEN activity_type in ('send','open') THEN time_spent ELSE 0 END) * 100
    , 2) AS send_perc,
    ROUND(SUM(CASE WHEN activity_type = 'open' THEN time_spent ELSE 0 END) / 
    SUM(CASE WHEN activity_type in ('send','open') THEN time_spent ELSE 0 END) * 100
    , 2)
AS open_perc
FROM cte
GROUP BY age_bucket
```




3. https://datalemur.com/questions/rolling-average-tweets

This is the same question as problem #10 in the SQL Chapter of Ace the Data Science Interview!

Given a table of tweet data over a specified time period, calculate the 3-day rolling average of tweets for each user. Output the user ID, tweet date, and rolling averages rounded to 2 decimal places.

Notes:
- A rolling average, also known as a moving average or running mean is a time-series technique that examines trends in data over a specified period of time.
- In this case, we want to determine how the tweet count for each user changes over a 3-day period.

`tweets` Table:
| Column Name | Type |
| --- | --- |
| user_id | integer |
| tweet_date | timestamp |
| tweet_count | integer |

```sql
SELECT 
    user_id, 
    tweet_date,
    CAST(AVG(tweet_count) OVER(
    PARTITION BY user_id
    ORDER BY tweet_date
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS NUMERIC(5,2)) AS three_days_rolling_avg 
FROM tweets;
```


4. https://datalemur.com/questions/sql-highest-grossing

Assume you're given a table containing data on Amazon customers and their spending on products in different category, write a query to identify the top two highest-grossing products within each category in the year 2022. The output should include the category, product, and total spend.

`product_spend` Table:
| Column Name |	Type |
| --- | --- |
| category | string |
| product | string |
| user_id | integer |
| spend | decimal |
| transaction_date | timestamp |

```sql
SELECT 
    category, 
    product, 
    total_spend
FROM
  (SELECT category, product, SUM(spend) as total_spend,
  RANK() OVER(
    PARTITION BY category
    ORDER BY SUM(spend) DESC
  ) AS ranking
  FROM product_spend
  WHERE EXTRACT(YEAR FROM transaction_date) = 2022
  GROUP BY category, product) AS subquery
WHERE ranking < 3
```


5. https://datalemur.com/questions/top-fans-rank

Assume there are three Spotify tables: artists, songs, and global_song_rank, which contain information about the artists, songs, and music charts, respectively.

Write a query to find the top 5 artists whose songs appear most frequently in the Top 10 of the global_song_rank table. Display the top 5 artist names in ascending order, along with their song appearance ranking.

If two or more artists have the same number of song appearances, they should be assigned the same ranking, and the rank numbers should be continuous (i.e. 1, 2, 2, 3, 4, 5). If you've never seen a rank order like this before, do the rank window function tutorial.

`artists` Table:
| Column Name | Type |
| --- | --- |
| artist_id | integer |
| artist_name | varchar |
| label_owner | varchar |

`songs` Table:
| Column Name | Type |
| --- | --- |
| song_id | integer |
| artist_id | integer |
| name | varchar |

`global_song_rank` Table:
| Column Name | Type |
| --- | --- |
| day | integer (1-52) |
| song_id | integer |
| rank | integer (1-1,000,000) |

```sql
with cte as 
    (SELECT a.artist_name, s.song_id, g.rank
    FROM artists as a
    JOIN songs as s 
    ON a.artist_id = s.artist_id
    JOIN global_song_rank as g
    ON s.song_id = g.song_id
    WHERE g.rank < 11)

SELECT 
    artist_name, 
    artist_rank
FROM
    (SELECT artist_name, 
    DENSE_RANK() OVER (
    ORDER BY COUNT(SONG_ID) DESC
    ) as artist_rank
    FROM cte
    GROUP BY artist_name) as subquery
WHERE artist_rank <= 5
```


6. https://datalemur.com/questions/signup-confirmation-rate

New TikTok users sign up with their emails. They confirmed their signup by replying to the text confirmation to activate their accounts. Users may receive multiple text messages for account confirmation until they have confirmed their new account.

A senior analyst is interested to know the activation rate of specified users in the emails table. Write a query to find the activation rate. Round the percentage to 2 decimal places.

Definitions:
- `emails` table contain the information of user signup details.
- `texts` table contains the users' activation information.

Assumptions:
- The analyst is interested in the activation rate of specific users in the `emails` table, which may not include all users that could potentially be found in the `texts` table.
- For example, user 123 in the `emails` table may not be in the `texts` table and vice versa.

`emails` Table:
| Column Name | Type |
| --- | --- |
| email_id | integer |
| user_id | integer |
| signup_date | datetime |

`texts` Table:
| Column Name | Type |
| --- | --- |
| text_id | integer |
| email_id | integer |
| signup_action | varchar |

```sql
WITH cte AS 
    (SELECT e.user_id,
    ROW_NUMBER() OVER (
    PARTITION BY user_id
    ORDER BY signup_date DESC
    ) as order_, *
    FROM emails as e
    JOIN texts as t
    ON e.email_id = t.email_id)

SELECT 
  ROUND(AVG((CASE 
  WHEN signup_action = 'Confirmed' 
  THEN (1.0/order_) 
  ELSE 0 END)), 2) AS confirm_rate
FROM cte
```


7. https://datalemur.com/questions/supercloud-customer

A Microsoft Azure Supercloud customer is defined as a company that purchases at least one product from each product category.

Write a query that effectively identifies the company ID of such Supercloud customers.

`customer_contracts` Table:
| Column Name | Type |
| --- | --- |
| customer_id | integer |
| product_id | integer |
| amount | integer |

`products` Table:
| Column Name | Type |
| --- | --- |
| product_id | integer |
| product_category | string |
| product_name | string |

```sql
SELECT 
    c.customer_id
FROM customer_contracts as c
JOIN products as p
ON c.product_id = p.product_id
GROUP BY c.customer_id
HAVING COUNT(DISTINCT product_category) > 2
```


8. https://datalemur.com/questions/odd-even-measurements

This is the same question as problem #28 in the SQL Chapter of Ace the Data Science Interview!

Assume you're given a table with measurement values obtained from a Google sensor over multiple days with measurements taken multiple times within each day.

Write a query to calculate the sum of odd-numbered and even-numbered measurements separately for a particular day and display the results in two different columns. Refer to the Example Output below for the desired format.

Definition:
- Within a day, measurements taken at 1st, 3rd, and 5th times are considered odd-numbered measurements, and measurements taken at 2nd, 4th, and 6th times are considered even-numbered measurements.

`measurements` Table:
| Column Name | Type |
| --- | --- |
| measurement_id | integer |
| measurement_value | decimal |
| measurement_time | datetime |

```sql
WITH cte AS 
    (SELECT *,
    measurement_time::date as measurement_day,
    ROW_NUMBER() OVER (
        PARTITION BY EXTRACT(DAY FROM measurement_time)
        ORDER BY measurement_time
    ) as order_
    FROM measurements)

SELECT 
measurement_day,
    SUM(CASE WHEN order_ % 2 = 1 THEN measurement_value	ELSE 0 END) as odd_sum,
    SUM(CASE WHEN order_ % 2 = 0 THEN measurement_value	ELSE 0 END) as even_sum
FROM cte
GROUP BY measurement_day
ORDER BY measurement_day
``` 


9. https://datalemur.com/questions/histogram-users-purchases

Assume you're given a table on Walmart user transactions. Based on their most recent transaction date, write a query that retrieve the users along with the number of products they bought.

Output the user's most recent transaction date, user ID, and the number of products, sorted in chronological order by the transaction date.

`user_transactions` Table:
| Column Name | Type |
| --- | --- |
| product_id | integer |
| user_id | integer |
| spend | decimal |
| transaction_date | timestamp |

```sql
SELECT 
    most_recent_date, 
    user_id, 
    COUNT(product_id) AS purchase_count
FROM
    (SELECT *,
    MAX(transaction_date) OVER(
    PARTITION BY user_id
    ORDER BY user_id
    ) as most_recent_date
    FROM user_transactions) AS SQ
WHERE transaction_date = most_recent_date
GROUP BY most_recent_date, user_id
```


10. https://datalemur.com/questions/alibaba-compressed-mode

You're given a table containing the item count for each order on Alibaba, along with the frequency of orders that have the same item count. Write a query to retrieve the mode of the order occurrences. Additionally, if there are multiple item counts with the same mode, the results should be sorted in ascending order.

Clarifications:
- item_count: Represents the number of items sold in each order.
- order_occurrences: Represents the frequency of orders with the corresponding number of items sold per order.
- For example, if there are 800 orders with 3 items sold in each order, the record would have an item_count of 3 and an order_occurrences of 800.

`items_per_order` Table:
| Column Name | Type |
| --- | --- |
| item_count | integer |
| order_occurrences | integer |

```sql
SELECT item_count
FROM items_per_order
WHERE order_occurrences = (SELECT MAX(order_occurrences)
FROM items_per_order
)
ORDER BY order_occurrences DESC
```


11. https://datalemur.com/questions/card-launch-success

Your team at JPMorgan Chase is soon launching a new credit card. You are asked to estimate how many cards you'll issue in the first month.

Before you can answer this question, you want to first get some perspective on how well new credit card launches typically do in their first month.

Write a query that outputs the name of the credit card, and how many cards were issued in its launch month. The launch month is the earliest record in the monthly_cards_issued table for a given card. Order the results starting from the biggest issued amount.

`monthly_cards_issued` Table:
| Column Name | Type |
| --- | --- |
| issue_month | integer |
| issue_year | integer |
| card_name | string |
| issued_amount | integer |

```sql
WITH cte AS 
    (SELECT card_name, issued_amount, issue_year, issue_month,
    RANK() OVER(
    PARTITION BY card_name
    ORDER BY issue_year, issue_month
    )
    FROM monthly_cards_issued)

SELECT card_name, issued_amount
FROM cte
WHERE rank = 1
ORDER BY issued_amount DESC
```


12. https://datalemur.com/questions/international-call-percentage

A phone call is considered an international call when the person calling is in a different country than the person receiving the call.

What percentage of phone calls are international? Round the result to 1 decimal.

Assumption:
- The caller_id in phone_info table refers to both the caller and receiver.

`phone_calls` Table:
| Column Name | Type |
| --- | --- |
| caller_id | integer |
| receiver_id | integer |
| call_time | timestamp |

`phone_info` Table:
| Column Name | Type |
| --- | --- |
| caller_id | integer |
| country_id | integer |
| network | integer |
| phone_number | string |

```sql
SELECT ROUND(SUM(is_international)*100/COUNT(is_international), 1) as international_calls_pct
FROM
(SELECT 
  pc.caller_id as caller_id, 
  pi1.country_id as caller_country,
  pc.receiver_id as receiver_id,
  pi2.country_id as receiver_country,
  CASE WHEN pi1.country_id != pi2.country_id THEN 1.0 ELSE 0.0 END as is_international
FROM phone_calls as pc
JOIN phone_info as pi1
ON pc.caller_id = pi1.caller_id
JOIN phone_info as pi2
ON pc.receiver_id = pi2.caller_id) AS sq
```

13. https://datalemur.com/questions/uncategorized-calls-percentage

UnitedHealth Group (UHG) has a program called Advocate4Me, which allows policy holders (or, members) to call an advocate and receive support for their health care needs – whether that's claims and benefits support, drug coverage, pre- and post-authorisation, medical records, emergency assistance, or member portal services.

Calls to the Advocate4Me call centre are classified into various categories, but some calls cannot be neatly categorised. These uncategorised calls are labeled as “n/a”, or are left empty when the support agent does not enter anything into the call category field.

Write a query to calculate the percentage of calls that cannot be categorised. Round your answer to 1 decimal place. For example, 45.0, 48.5, 57.7.

`callers` Table:
| Column Name | Type |
| --- | --- |
| policy_holder_id | integer |
| case_id | varchar |
| call_category | varchar |
| call_date | timestamp |
| call_duration_secs | integer |

```sql
SELECT 
  ROUND(SUM(CASE WHEN call_category is NULL THEN 1.0
  WHEN call_category = 'n/a' THEN 1.0
  ELSE 0.0 END) * 100 / COUNT(case_id), 1) AS uncategorised_call_pct
FROM callers;
```
